ESTE ES EL FINAL, GODINES

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

# Define la carpeta local
carpeta_local = '/content/drive/MyDrive/CapstoneData/Geisha'

# Listas para almacenar las métricas de todos los archivos
rmse_pls_list = []
mae_pls_list = []
precision_pls_list = []

rmse_rf_list = []
mae_rf_list = []
precision_rf_list = []

rmse_svm_list = []
mae_svm_list = []
precision_svm_list = []

# Contador para limitar a 15 archivos
contador_archivos = 0

# Bucle para procesar archivos en la carpeta
for nombre_archivo in os.listdir(carpeta_local):
    if contador_archivos >= 15:
        break  # Sal del bucle si ya se leyeron 15 archivos

    if nombre_archivo.endswith("_a.csv"):
        # Lee el archivo CSV y salta las primeras 28 filas
        ruta_completa = os.path.join(carpeta_local, nombre_archivo)
        df = pd.read_csv(ruta_completa, skiprows=28)

        # Detección y manejo de valores atípicos
        Q1 = df['Absorbance (AU)'].quantile(0.25)
        Q3 = df['Absorbance (AU)'].quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((df['Absorbance (AU)'] < (Q1 - 1.5 * IQR)) | (df['Absorbance (AU)'] > (Q3 + 1.5 * IQR)))

        # Opción 1: Eliminar valores atípicos
        df_sin_atipicos = df[~outliers]

        # Opción 2: Ajustar valores atípicos
        df_ajustado = df.copy()
        df_ajustado.loc[outliers, 'Absorbance (AU)'] = df['Absorbance (AU)'].median()

        # División de datos en conjunto de entrenamiento y prueba (80-20)
        X = df_sin_atipicos['Wavelength (nm)'].values.reshape(-1, 1)
        y = df_sin_atipicos['Absorbance (AU)'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Entrenamiento del modelo PLS con 1 componente
        pls_model = PLSRegression(n_components=1)
        pls_model.fit(X_train, y_train)

        # Predicciones en el conjunto de prueba para PLS
        y_pred_pls = pls_model.predict(X_test)

        # Calcula RMSE y MAE para PLS
        rmse_pls = sqrt(mean_squared_error(y_test, y_pred_pls))
        mae_pls = mean_absolute_error(y_test, y_pred_pls)

        # Calcula la precisión para PLS en porcentaje
        rango_y = np.max(y) - np.min(y)
        precision_pls = (1 - rmse_pls / rango_y) * 100

        # Almacena las métricas en las listas
        rmse_pls_list.append(rmse_pls)
        mae_pls_list.append(mae_pls)
        precision_pls_list.append(precision_pls)

        # Entrenamiento del modelo de Bosques Aleatorios
        rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_model.fit(X_train, y_train)

        # Predicciones en el conjunto de prueba para Bosques Aleatorios
        y_pred_rf = rf_model.predict(X_test)

        # Calcula RMSE y MAE para Bosques Aleatorios
        rmse_rf = sqrt(mean_squared_error(y_test, y_pred_rf))
        mae_rf = mean_absolute_error(y_test, y_pred_rf)

        # Calcula la precisión para Bosques Aleatorios
        precision_rf = (1 - rmse_rf / rango_y) * 100

        # Almacena las métricas en las listas
        rmse_rf_list.append(rmse_rf)
        mae_rf_list.append(mae_rf)
        precision_rf_list.append(precision_rf)

        # Entrenamiento del modelo de Máquinas de Vectores de Soporte (SVM)
        svm_model = SVR(kernel='linear')
        svm_model.fit(X_train, y_train)

        # Predicciones en el conjunto de prueba para SVM
        y_pred_svm = svm_model.predict(X_test)

        # Calcula RMSE y MAE para SVM
        rmse_svm = sqrt(mean_squared_error(y_test, y_pred_svm))
        mae_svm = mean_absolute_error(y_test, y_pred_svm)

        # Calcula la precisión para SVM
        precision_svm = (1 - rmse_svm / rango_y) * 100

        # Almacena las métricas en las listas
        rmse_svm_list.append(rmse_svm)
        mae_svm_list.append(mae_svm)
        precision_svm_list.append(precision_svm)

        # Imprime las métricas
        print(f"Archivo: {nombre_archivo}")
        print(f"RMSE (PLS): {rmse_pls}, MAE (PLS): {mae_pls}")
        print(f"Precisión (PLS): {precision_pls}%")
        print(f"RMSE (Bosques Aleatorios): {rmse_rf}, MAE (Random Forest): {mae_rf}")
        print(f"Precisión (Bosques Aleatorios): {precision_rf}%")
        print(f"RMSE (SVM): {rmse_svm}, MAE (SVM): {mae_svm}")
        print(f"Precisión (SVM): {precision_svm}%")
        print(f"------------------------------------------------------------------------------")



        # Guardar el DataFrame limpio o ajustado si es necesario
        df_sin_atipicos.to_csv(os.path.join(carpeta_local, nombre_archivo.replace(".csv", "_limpio.csv")), index=False)

        contador_archivos += 1  # Incrementa el contador después de procesar un archivo

# Calcula el promedio de las métricas para PLS
promedio_rmse_pls = sum(rmse_pls_list) / len(rmse_pls_list)
promedio_mae_pls = sum(mae_pls_list) / len(mae_pls_list)
promedio_precision_pls = sum(precision_pls_list) / len(precision_pls_list)

print(f"Promedio de RMSE (PLS): {promedio_rmse_pls}, Promedio de MAE (PLS): {promedio_mae_pls}, Promedio de Precisión (PLS): {promedio_precision_pls}%")

# Calcula el promedio de las métricas para Bosques Aleatorios
promedio_rmse_rf = sum(rmse_rf_list) / len(rmse_rf_list)
promedio_mae_rf = sum(mae_rf_list) / len(mae_rf_list)
promedio_precision_rf = sum(precision_rf_list) / len(precision_rf_list)

print(f"Promedio de RMSE (Bosques Aleatorios): {promedio_rmse_rf}, Promedio de MAE (Bosques Aleatorios): {promedio_mae_rf}, Promedio de Precisión (Bosques Aleatorios): {promedio_precision_rf}%")

# Calcula el promedio de las métricas para SVM
promedio_rmse_svm = sum(rmse_svm_list) / len(rmse_svm_list)
promedio_mae_svm = sum(mae_svm_list) / len(mae_svm_list)
promedio_precision_svm = sum(precision_svm_list) / len(precision_svm_list)

print(f"Promedio de RMSE (SVM): {promedio_rmse_svm}, Promedio de MAE (SVM): {promedio_mae_svm}, Promedio de Precisión (SVM): {promedio_precision_svm}%")


Archivo: Hadamard 1_20230916_153715_a.csv
RMSE (PLS): 0.055310846812090156, MAE (PLS): 0.049795194483757285
Precisión (PLS): 70.42723658263759%
RMSE (Bosques Aleatorios): 0.002911597905710347, MAE (Random Forest): 0.0016652447605203084
Precisión (Bosques Aleatorios): 98.44327105812383%
RMSE (SVM): 0.06186210810314238, MAE (SVM): 0.054538477001924913
Precisión (SVM): 66.92450770734466%
------------------------------------------------------------------------------
Archivo: Hadamard 1_20230916_153735_a.csv
RMSE (PLS): 0.05314547077204601, MAE (PLS): 0.0482539444470051
Precisión (PLS): 70.19324570182017%
RMSE (Bosques Aleatorios): 0.004872565530760233, MAE (Random Forest): 0.0023978036335355222
Precisión (Bosques Aleatorios): 97.26721089366015%
RMSE (SVM): 0.05908019846468923, MAE (SVM): 0.05150268474938904
Precisión (SVM): 66.86474060831999%
------------------------------------------------------------------------------
Archivo: Hadamard 1_20230916_153808_a.csv
RMSE (PLS): 0.05632977512537

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
